In [2]:
!pip install sqlalchemy langchain langchain-community langchain-openai shap transformers bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.75
    Uninstalling langchain-core-0.3.75:
      Successfully uninstalled langchain-core-0.3.75
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googl

In [ ]:
# Import necessary libraries (updated with missing imports)
from google.colab import userdata
import os
import json
import re
import numpy as np
import pandas as pd
import sqlite3
import joblib
import shap
from typing import Dict, Any
from pydantic import BaseModel, Field

# LangChain imports (updated)
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent, AgentType
from langchain_core.messages import HumanMessage, AIMessage
from langchain.agents import tool

# Load API keys
hf_token = userdata.get('HF_TOKEN')
openai_api_key = userdata.get('openai_api')
openrouter_base = userdata.get('openai_base_url')
MODEL_NAME = "gpt-4o-mini"

In [155]:
# Set up the LLM (temperature 0 for determinism)
llm = ChatOpenAI(
    model=MODEL_NAME,
    openai_api_key=openai_api_key,
    base_url=openrouter_base,
    temperature=0.0
)

In [156]:
# Load dataset into SQLite
df = pd.read_csv('/content/data/dataset.csv')
conn = sqlite3.connect('customers.db')
df.to_sql('customers', conn, if_exists='replace', index=False)

# Print sample to verify
pd.read_sql('SELECT * FROM customers LIMIT 5', conn)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [97]:
# Load model and preprocessor
model = joblib.load('/content/assets/Tuned-RF-with-SMOTE.pkl')
preprocessor = joblib.load('/content/assets/preprocessor.pkl')
explainer = shap.TreeExplainer(model)

In [112]:
# Pydantic model (unchanged)
class CustomerData(BaseModel):
    CreditScore: float = Field(..., description='Credit score of the customer')
    Geography: str = Field(..., description='Geography')
    Gender: str = Field(..., description='Gender')
    Age: int = Field(..., description='Age of the customer')
    Tenure: int = Field(..., description='Number of years the customer has been with the bank')
    Balance: float = Field(..., description='Account balance')
    NumOfProducts: int = Field(..., description='Number of products the customer has')
    HasCrCard: bool = Field(..., description='Does the customer have a credit card (True for yes, False for no)')
    IsActiveMember: bool = Field(..., description='Is the customer an active member (True for yes, False for no)')
    EstimatedSalary: float = Field(..., description='Estimated salary of the customer')

In [171]:
# Text extraction (unchanged)
def generate_prompt(text: str) -> str:
    return f"""
Extract the following fields from the text and provide them in JSON format:
CreditScore, Geography, Gender, Age, Tenure, Balance, NumOfProducts, HasCrCard, IsActiveMember, EstimatedSalary.

Rules:
- All values must be in English and numeric where applicable.
- For boolean fields, use true/false.

Example:
Text: "Jane Smith is a 35-year-old female from Canada with a credit score of 650. She has been with the bank for 3 years, has a balance of 2000.0 USD, holds 1 product, owns a credit card, is an active member, and earns an estimated salary of 75000.0 USD."
JSON: {{
    "CreditScore": 650,
    "Geography": "Canada",
    "Gender": "Female",
    "Age": 35,
    "Tenure": 3,
    "Balance": 2000.0,
    "NumOfProducts": 1,
    "HasCrCard": true,
    "IsActiveMember": true,
    "EstimatedSalary": 75000.0
}}

Text: "{text}"
JSON:
"""

def parse_text_to_json(text: str) -> Dict[str, Any]:
    prompt = generate_prompt(text)
    response = llm.invoke(prompt).content
    json_match = re.search(r'\{.*\}', response, re.DOTALL)
    if json_match:
        try:
            data = json.loads(json_match.group(0))
            data['HasCrCard'] = bool(data.get('HasCrCard', False))
            data['IsActiveMember'] = bool(data.get('IsActiveMember', False))
            return data
        except:
            pass
    return None

In [ ]:
# Test extraction
sample_text = """
Mohammed is a 27-year-old male from Spain with a credit score of 700. He has been with the bank for 5 years, has a balance of 5000.0 USD, holds 2 products, owns a credit card, is an active member, and earns an estimated salary of 100000.0 USD.
"""
parsed = parse_text_to_json(sample_text)
print(parsed)  # Should print dict

{'CreditScore': 700, 'Geography': 'Spain', 'Gender': 'Male', 'Age': 27, 'Tenure': 5, 'Balance': 5000.0, 'NumOfProducts': 2, 'HasCrCard': True, 'IsActiveMember': True, 'EstimatedSalary': 100000.0}


In [158]:
# SQL setup (unchanged)
db = SQLDatabase.from_uri("sqlite:///customers.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [157]:
sql_agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    max_iterations=5
)

In [159]:
# Chains (unchanged prompts, but will fix access later)
explain_prompt = PromptTemplate(
    template="""Generate a natural language explanation for the churn prediction.
Prediction (0=no churn, 1=churn): {pred}
Churn Probability: {prob:.2f}
SHAP values: {shap}
Customer data: {data}

Explain in clear, user-friendly language why the model predicted this, highlighting key factors.
    """,
    input_variables=["pred", "prob", "shap", "data"]
)
explain_chain = LLMChain(llm=llm, prompt=explain_prompt)

recommend_prompt = PromptTemplate(
    template="""Based on the customer data: {data}
Recommend actions or products to reduce churn risk or engage the customer.
Provide 3-5 specific recommendations in bullet points.
    """,
    input_variables=["data"]
)
recommend_chain = LLMChain(llm=llm, prompt=recommend_prompt)

In [160]:
# Tools with fixes for SHAP access and chain output

@tool
def prediction_tool(query: str) -> str:
    """Use for predicting churn for new or existing customers. Input can be free text or Customer ID."""
    if re.search(r'\d{8}', query):
        customer_id = re.search(r'\d{8}', query).group(0)
        customer_df = pd.read_sql(f"SELECT * FROM customers WHERE CustomerId = {customer_id}", conn)
        if customer_df.empty:
            return "Customer not found."
        features = customer_df.iloc[0].to_dict()
    else:
        features = parse_text_to_json(query)
        if not features:
            return "Invalid customer data."

    df = pd.DataFrame([features])
    processed = preprocessor.transform(df)
    pred = model.predict(processed)[0]
    prob = model.predict_proba(processed)[0][1]
    shap_values = explainer.shap_values(processed)
    # Fix: conditional access for SHAP values
    if isinstance(shap_values, list):
        shap_vals = shap_values[1][0]
    else:
        shap_vals = shap_values[0]
    feature_names = preprocessor.get_feature_names_out()
    shap_dict = dict(zip(feature_names, shap_vals))

    # Fix: use ['text'] for chain output
    explanation = explain_chain.invoke({
        "pred": pred,
        "prob": prob,
        "shap": shap_dict,
        "data": features
    })['text']
    return explanation + f"\nPrediction: {pred}, Prob: {prob:.2f}"

@tool
def sql_tool(query: str) -> str:
    """Use for aggregate queries or fetching data from the database."""
    result = sql_agent.invoke(query)
    return result['output']

@tool
def recommendation_tool(query: str) -> str:
    """Recommend actions based on customer features. Input text or ID."""
    if re.search(r'\d{8}', query):
        customer_id = re.search(r'\d{8}', query).group(0)
        customer_df = pd.read_sql(f"SELECT * FROM customers WHERE CustomerId = {customer_id}", conn)
        if customer_df.empty:
            return "Customer not found."
        features = customer_df.iloc[0].to_dict()
    else:
        features = parse_text_to_json(query)
        if not features:
            return "Invalid customer data."

    # Fix: use ['text']
    recommendations = recommend_chain.invoke({"data": features})['text']
    return recommendations

In [161]:
# Fixed Router Agent with better prompt and follow-up handling
def router_agent(query: str, chat_history: list = []) -> str:
    history_str = "\n".join([f"{msg['role']}: {msg['content']}" for msg in chat_history[-4:]])

    router_prompt = PromptTemplate(
        template="""
Previous conversation:
{history}

Current query: {query}

Classify the query and choose ONE tool:
- prediction_tool if the query is about predicting churn or explaining a prediction (keywords: predict, churn, explain, why).
- recommendation_tool if the query is about recommendations, actions, or products to retain the customer (keywords: recommend, actions, suggestions).
- sql_tool if the query is about database queries, aggregates, averages, counts, lists, or showing multiple customers (keywords: average, count, show all, list, how many).

Examples:
- "Predict churn for customer ID 12345678": prediction_tool
- "Recommend actions for this customer": recommendation_tool
- "What is the average age of exited customers?": sql_tool
- "Show customers with Balance > 100000": sql_tool

Respond ONLY with the tool name: prediction_tool, recommendation_tool, or sql_tool.
        """,
        input_variables=["history", "query"]
    )
    router_chain = LLMChain(llm=llm, prompt=router_prompt)

    response = router_chain.invoke({"history": history_str, "query": query})
    tool_name = response['text'].strip().lower()

    # Handle follow-up references to 'this customer'
    tool_input = query
    if 'this customer' in query.lower() or 'the customer' in query.lower():
        for msg in reversed(chat_history):
            if msg['role'] == 'user' and (re.search(r'\d{8}', msg['content']) or 'year-old' in msg['content']):
                tool_input = msg['content']  # Use previous data query
                break

    if "prediction" in tool_name:
        return prediction_tool(tool_input)
    elif "recommendation" in tool_name:
        return recommendation_tool(tool_input)
    elif "sql" in tool_name:
        return sql_tool(tool_input)
    else:
        return "Unable to route query. Please rephrase."

chat_history = []

### *New customer prediction*

In [121]:
query = "Maher is a 28-year-old male from Spain with a credit score of 700. He has been with the bank for 5 years, has a balance of 5000.0, holds 2 products, owns a credit card, is an active member, and earns an estimated salary of 100000.0. Predict his churn."
response = router_agent(query, chat_history)
print(response)
chat_history.append({"role": "user", "content": query})
chat_history.append({"role": "assistant", "content": response})

The churn prediction model has determined that this customer is unlikely to leave, with a churn probability of just 9%. This means that there is a strong likelihood that they will continue to use the service. Let's break down the key factors that contributed to this prediction:

1. **Age**: The customer is 28 years old, and this age group tends to show lower churn rates. The model indicates that being in this age range positively influences their likelihood to stay.

2. **Credit Score**: With a credit score of 700, this customer is considered to have a good credit standing. While the impact of credit score on churn is relatively small, it still contributes positively to the prediction.

3. **Balance**: The customer has a balance of $5,000. Interestingly, the model suggests that a higher balance can be associated with lower churn, although in this case, the effect is not very strong.

4. **Number of Products**: The customer holds 2 products with the company. The model shows that having 

### *Follow-up recommendation*

In [122]:
query = "Now recommend actions for this customer."
response = router_agent(query, chat_history)
print(response)
chat_history.append({"role": "user", "content": query})
chat_history.append({"role": "assistant", "content": response})

Based on the provided customer data, here are several recommendations to reduce churn risk and engage the customer:

- **Personalized Financial Products**: Offer tailored financial products such as a premium credit card or a savings account with higher interest rates. Given the customer's good credit score and active status, they may appreciate exclusive benefits that enhance their banking experience.

- **Loyalty Rewards Program**: Introduce a loyalty program that rewards the customer for their tenure and engagement with the bank. This could include cashback on purchases, discounts on fees, or points that can be redeemed for travel or shopping.

- **Financial Advisory Services**: Provide access to personalized financial advisory services. Given the customer's age and salary, they may benefit from investment advice or retirement planning, which can help them feel more valued and engaged with the bank.

- **Engagement through Educational Content**: Create and share educational content r

### *Existing customer by ID*

In [123]:
query = "Predict churn for customer ID 15634602"
response = router_agent(query, chat_history)
print(response)
chat_history.append({"role": "user", "content": query})
chat_history.append({"role": "assistant", "content": response})

The model has predicted that this customer is likely to churn, with a churn probability of 58%. This means there is a significant chance that they will stop using the service. Let's break down the key factors that contributed to this prediction:

1. **Age**: The customer's age of 42 had a slight negative impact on their likelihood to churn. This suggests that, generally, older customers may be less likely to leave, but in this case, it didn't play a major role.

2. **Credit Score**: The customer's credit score of 619 is relatively average. The model indicates that this factor had a minimal effect on the churn prediction, meaning it wasn't a strong indicator of their likelihood to leave.

3. **Balance**: The customer has a balance of $0.00. This is a concerning factor, as having no balance can indicate a lack of engagement or investment in the service, which can lead to higher churn rates.

4. **Tenure**: The customer has been with the service for only 2 years. Shorter tenure often corr

### *Aggregate SQL query*

In [124]:
query = "Show me the average age of customers who exited (Exited=1)"
response = router_agent(query, chat_history)
print(response)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  customersI need to check the schema of the "customers" table to find the relevant columns for calculating the average age of customers who exited.  
Action: sql_db_schema  
Action Input: "customers"  
CREATE TABLE customers (
	"RowNumber" INTEGER, 
	"CustomerId" INTEGER, 
	"Surname" TEXT, 
	"CreditScore" INTEGER, 
	"Geography" TEXT, 
	"Gender" TEXT, 
	"Age" INTEGER, 
	"Tenure" INTEGER, 
	"Balance" REAL, 
	"NumOfProducts" INTEGER, 
	"HasCrCard" INTEGER, 
	"IsActiveMember" INTEGER, 
	"EstimatedSalary" REAL, 
	"Exited" INTEGER
)

/*
3 rows from customers table:
RowNumber	CustomerId	Surname	CreditScore	Geography	Gender	Age	Tenure	Balance	NumOfProducts	HasCrCard	IsActiveMember	EstimatedSalary	Exited
1	15634602	Hargrave	619	France	Female	42	2	0.0	1	1	1	101348.88	1
2	15647311	Hill	608	Spain	Female	41	1	83807.86	1	0	1	112542.58	0
3	15619304	Onio	502	France	Female	42	8	159660.8	3	1	0	113931.57	1
*/I can 

### *High churn prob (manual, as before)*

In [125]:
all_customers = pd.read_sql("SELECT * FROM customers", conn)
features_df = all_customers[['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']]
processed_all = preprocessor.transform(features_df)
probs = model.predict_proba(processed_all)[:, 1]
all_customers['churn_prob'] = probs
high_churn = all_customers[all_customers['churn_prob'] > 0.7]
print(high_churn[['CustomerId', 'Surname', 'churn_prob']].to_markdown())

|      |   CustomerId | Surname             |   churn_prob |
|-----:|-------------:|:--------------------|-------------:|
|    2 |     15619304 | Onio                |     0.877592 |
|    7 |     15656148 | Obinna              |     0.790047 |
|   16 |     15737452 | Romeo               |     0.912032 |
|   30 |     15589475 | Azikiwe             |     0.759479 |
|   35 |     15794171 | Lombardo            |     0.732907 |
|   41 |     15738148 | Clarke              |     0.795629 |
|   43 |     15755196 | Lavine              |     0.777252 |
|   54 |     15569590 | Yoo                 |     0.835176 |
|   58 |     15623944 | T'ien               |     0.72471  |
|   70 |     15703793 | Konovalova          |     0.945706 |
|   88 |     15622897 | Sharpe              |     0.878121 |
|   90 |     15757535 | Heap                |     0.886021 |
|  127 |     15782688 | Piccio              |     0.905621 |
|  140 |     15698932 | Groves              |     0.814909 |
|  143 |     15713483 | 

### *Aggregate SQL query*

In [127]:
query = "show me all Customer with high balance but low activity"
response = router_agent(query, chat_history)
print(response)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  customersI need to check the schema of the "customers" table to understand its structure and identify the relevant columns for balance and activity.  
Action: sql_db_schema  
Action Input: "customers"  
CREATE TABLE customers (
	"RowNumber" INTEGER, 
	"CustomerId" INTEGER, 
	"Surname" TEXT, 
	"CreditScore" INTEGER, 
	"Geography" TEXT, 
	"Gender" TEXT, 
	"Age" INTEGER, 
	"Tenure" INTEGER, 
	"Balance" REAL, 
	"NumOfProducts" INTEGER, 
	"HasCrCard" INTEGER, 
	"IsActiveMember" INTEGER, 
	"EstimatedSalary" REAL, 
	"Exited" INTEGER
)

/*
3 rows from customers table:
RowNumber	CustomerId	Surname	CreditScore	Geography	Gender	Age	Tenure	Balance	NumOfProducts	HasCrCard	IsActiveMember	EstimatedSalary	Exited
1	15634602	Hargrave	619	France	Female	42	2	0.0	1	1	1	101348.88	1
2	15647311	Hill	608	Spain	Female	41	1	83807.86	1	0	1	112542.58	0
3	15619304	Onio	502	France	Female	42	8	159660.8	3	1	0	113931.57	1
*/To i

### *Another Aggregate SQL query*

In [128]:
query = "Show me all customers from France with churn probability greater than 0.7"
response = router_agent(query, chat_history)
print(response)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  customersI need to check the schema of the "customers" table to find the relevant columns for my query.  
Action: sql_db_schema  
Action Input: "customers"  
CREATE TABLE customers (
	"RowNumber" INTEGER, 
	"CustomerId" INTEGER, 
	"Surname" TEXT, 
	"CreditScore" INTEGER, 
	"Geography" TEXT, 
	"Gender" TEXT, 
	"Age" INTEGER, 
	"Tenure" INTEGER, 
	"Balance" REAL, 
	"NumOfProducts" INTEGER, 
	"HasCrCard" INTEGER, 
	"IsActiveMember" INTEGER, 
	"EstimatedSalary" REAL, 
	"Exited" INTEGER
)

/*
3 rows from customers table:
RowNumber	CustomerId	Surname	CreditScore	Geography	Gender	Age	Tenure	Balance	NumOfProducts	HasCrCard	IsActiveMember	EstimatedSalary	Exited
1	15634602	Hargrave	619	France	Female	42	2	0.0	1	1	1	101348.88	1
2	15647311	Hill	608	Spain	Female	41	1	83807.86	1	0	1	112542.58	0
3	15619304	Onio	502	France	Female	42	8	159660.8	3	1	0	113931.57	1
*/The "customers" table contains relevant columns, 

In [173]:
query_ar = (
    "عمرو عمره 35 سنة من اسبانيا، عنده درجة ائتمان 750. "
    "مستمر مع البنك لمدة 4 سنوات، رصيده 12000.0، "
    "يمتلك 3 منتجات، لديه بطاقة ائتمان، عضو نشط، "
    "ودخله التقديري 150000.0. توقع نسبة احتمالية تركه للخدمة (churn)."
)

response_ar = router_agent(query_ar, chat_history)
print(response_ar)

# تحديث سجل المحادثة
chat_history.append({"role": "user", "content": query_ar})
chat_history.append({"role": "assistant", "content": response_ar})

The model has predicted that this customer is likely to churn, with a churn probability of 54%. This means there is a slightly more than even chance that they will stop using the service. Let's break down the key factors that contributed to this prediction:

1. **Number of Products**: The most significant factor influencing the churn prediction is the number of products the customer has. In this case, the customer has three products, which is associated with a higher likelihood of churn. This suggests that customers with fewer products may feel less engaged or satisfied, leading them to consider leaving.

2. **Is Active Member**: The customer is marked as an active member, which typically indicates engagement. However, while this is a positive factor, it wasn't strong enough to outweigh the negative impact of having three products.

3. **Age**: The customer's age of 35 has a slight positive influence on the churn prediction. Generally, younger customers may be more likely to switch ser

In [174]:
# Example Test Cell 2: Follow-up recommendation
query = "دلوقتي اعطيني توصيات للتصرف مع هذا العميل."
response = router_agent(query, chat_history)
print(response)
chat_history.append({"role": "user", "content": query})
chat_history.append({"role": "assistant", "content": response})

Based on the provided customer data, here are some recommendations to reduce churn risk and engage the customer:

- **Personalized Communication**: Initiate a personalized outreach campaign to understand the customer's needs and preferences. Use surveys or direct communication to gather information about their interests and concerns, which can help tailor future interactions.

- **Incentivize Engagement**: Offer incentives for the customer to engage with your services, such as a small bonus for completing a survey or a discount on their next transaction. This can help increase their interaction with your brand.

- **Educational Content**: Provide educational resources or workshops that highlight the benefits of your products and services. This could include financial literacy content, tips on managing credit, or information on how to maximize the use of their current products.

- **Loyalty Program Introduction**: Introduce a loyalty program that rewards customers for their engagement a

In [175]:
# Example Test Cell 3: Existing customer by ID
query = "توقع احتمالية ترك العميل ذو الرقم التعريفي 15634602 للبنك."
response = router_agent(query, chat_history)
print(response)
chat_history.append({"role": "user", "content": query})
chat_history.append({"role": "assistant", "content": response})

The model has predicted that this customer is likely to churn, with a churn probability of 58%. This means there is a significant chance that they will stop using the service. Let's break down the key factors that contributed to this prediction:

1. **Age**: The customer's age of 42 had a slight negative impact on their likelihood to churn. This suggests that, generally, older customers may be less likely to leave, but in this case, it didn't play a strong role.

2. **Credit Score**: The customer's credit score of 619 is relatively average. The model indicates that this factor had a minimal effect on the churn prediction, meaning it wasn't a major reason for the likelihood of leaving.

3. **Balance**: The customer has a balance of $0.00. This is a concerning factor, as having no balance can indicate a lack of engagement with the service, which may lead to higher chances of churn.

4. **Tenure**: The customer has been with the service for only 2 years. Shorter tenure can often correlate

In [176]:
# Example Test Cell 4: Aggregate SQL query
query = "اعرض لي كل العملاء من اسبانيا الذين لديهم احتمالية ترك البنك أكبر من 0.7"
response = router_agent(query, chat_history)
print(response)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  customersI need to check the schema of the "customers" table to find the relevant columns for my query.  
Action: sql_db_schema  
Action Input: "customers"  
CREATE TABLE customers (
	"RowNumber" INTEGER, 
	"CustomerId" INTEGER, 
	"Surname" TEXT, 
	"CreditScore" INTEGER, 
	"Geography" TEXT, 
	"Gender" TEXT, 
	"Age" INTEGER, 
	"Tenure" INTEGER, 
	"Balance" REAL, 
	"NumOfProducts" INTEGER, 
	"HasCrCard" INTEGER, 
	"IsActiveMember" INTEGER, 
	"EstimatedSalary" REAL, 
	"Exited" INTEGER
)

/*
3 rows from customers table:
RowNumber	CustomerId	Surname	CreditScore	Geography	Gender	Age	Tenure	Balance	NumOfProducts	HasCrCard	IsActiveMember	EstimatedSalary	Exited
1	15634602	Hargrave	619	France	Female	42	2	0.0	1	1	1	101348.88	1
2	15647311	Hill	608	Spain	Female	41	1	83807.86	1	0	1	112542.58	0
3	15619304	Onio	502	France	Female	42	8	159660.8	3	1	0	113931.57	1
*/I need to construct a query to find customers fro